## Projeto
Classificação binária - Real ou Deepfake.

---

Link do desafio no Kaggle:
https://www.kaggle.com/competitions/image-classification-real-or-ai-generated-photo/data

In [ ]:
import os
import pandas as pd

### Open and investigate Dataset

In [ ]:
from pathlib import Path
notebook_path = Path().resolve()

dataset_base_path = notebook_path.parent / "dataset/"

train_path = dataset_base_path / "train/train"
test_path = dataset_base_path / "test/test"

train_images = sorted(os.listdir(train_path), key=lambda x: int(os.path.splitext(x)[0]))
test_images = sorted(os.listdir(test_path), key=lambda x: int(os.path.splitext(x)[0]))

train_labels = pd.read_csv(dataset_base_path / "train.csv")
test_labels = pd.read_csv(dataset_base_path / "test.csv")

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import random

In [ ]:
def print_images(label, sample_images):
    fig, axes = plt.subplots(2, 3, figsize=(12, 8))
    for ax, img_name in zip(axes.flatten(), sample_images):
        img = Image.open(train_path / img_name)
        ax.imshow(img)
        ax.set_title(img_name)
        ax.axis('off')

    plt.tight_layout()
    plt.show()


In [ ]:
label_ai_generated = 0
label_real_images = 1

real_train_images = train_labels[train_labels['Label'] == label_real_images]['Image'].tolist()
fake_train_images = train_labels[train_labels['Label'] == label_ai_generated]['Image'].tolist()

print_images(label_ai_generated, random.sample(real_train_images, min(6, len(real_train_images))))
print_images(label_real_images, random.sample(fake_train_images, min(6, len(fake_train_images))))

In [ ]:
# Imprimir imagens das flores

# Dividir em treino e validação (teste esta separado)

In [ ]:
# (pré-processamento, normalização, validação, modelagem, comparação).